<a href="https://colab.research.google.com/github/selmanbaysan/github-slideshow/blob/master/python_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tkinter import * # this code import the tkinter module
from tkinter.ttk import Combobox, Treeview # this code import combobox, treeview mode
from selenium import webdriver # this code import selenium mdule
from bs4 import BeautifulSoup # this code import beautiful soup module
import requests # this code import requests module
import mysearchengine # this code import mysearchengine python file
import os # this code import os module


class GUI(Frame): #this code creates frame
    def __init__(self, parent):
        Frame.__init__(self, parent) # this code ensure that the code we write is in the frame
        self.root = parent
        self.initUI()

    def initUI(self):

        self.mainFrame = Frame(self, bg="light grey") # this code determines the properties of the main frame
        self.mainFrame.pack(fill=BOTH, expand=True) # this code creates the main frame

        self.headLabel = Label(self.mainFrame, text="Sehir Faculty Member Search Engine" ,bg="blue", fg="white", font=("Helvetica",30, "bold")) # this code determines the properties of the head label
        self.headLabel.pack(fill=X) # this code creates the head label

        self.urlFrame = Frame(self.mainFrame, bg="light grey") # this code determines the properties of the url frame
        self.urlFrame.pack() # this code creates the url frame

        self.urlLabel = Label(self.urlFrame, text="Faculty Profile URL:", bg="light grey", font=("Helvetica", 15)) # this code determines the properties of the url label
        self.urlLabel.pack(side=LEFT) # this code creates the url label

        self.urlEntry = Entry(self.urlFrame) # this code determines the properties of the url entry
        self.urlEntry.insert(END, "https://www.sehir.edu.tr/en/academics/college-of-engineering-and-natural-sciences/academic-staff") # this code insert the url into the url entry
        self.urlEntry.pack(side=LEFT, ipadx=250) # this code creates the entry

        self.fetchButton = Button(self.mainFrame, text="Fetch Profiles", bg="light grey") # this code determines the properties of the fetch button
        self.fetchButton.pack(pady=20) # this code creates the fetch button
        self.fetchButton.bind("<Button-1>", self.fetch_profiles) # this code make  fetch_profiles func event on fetch button

        self.progressFrame = Frame(self.mainFrame, bg="light grey") # this code determines the properties of the progress frame
        self.progressFrame.pack() # this code creates the progress frame

        self.progressLabel = Label(self.progressFrame, bg="light grey") # this code determines the properties of the progress label
        self.progressLabel.pack(side=LEFT) # this code creates the progress label

        self.progressBar = Canvas(self.progressFrame,bg="gray", borderwidth=2,  width=400, height=15, relief="flat") # this code determines the properties of the progressbar canvas
        self.progressBar.pack(side=LEFT, padx=40) # this code creates the progressbar canvas

        self.persentaceLabel = Label(self.progressFrame, bg="light grey", text="0% Completed") # this code determines the properties of the percentage label
        self.persentaceLabel.pack(side=LEFT) # this code creates the percentage label

        self.keydepFrame = Frame(self.mainFrame, bg="light grey") # this code determines the properties of the keydep frame
        self.keydepFrame.pack(pady=50) # this code creates the keydep frame

        self.keywordLabel = Label(self.keydepFrame, bg="light grey", text="Keywords:", font=("Helvetica", 20)) # this code determines the properties of the keyword label
        self.keywordLabel.pack(side=LEFT, padx=30) # this code creates the keyword label

        self.keywordEntry = Entry(self.keydepFrame) # this code determines the properties of the keyword entry
        self.keywordEntry.pack(side=LEFT, padx=20, ipadx=50) # this code creates the keyword entry

        self.departmentLabel = Label(self.keydepFrame, text="Department:", bg="light grey",font=("Helvetica", 20)) # this code determines the properties of the department label
        self.departmentLabel.pack(side=LEFT, padx=20) # this code creates the department label

        self.departmentCombobox = Combobox(self.keydepFrame) # this code determines the properties of the department combobox
        self.departmentCombobox.pack(side=LEFT, padx=30) # this code creates the combobox

        self.searchButton = Button(self.mainFrame, text="Search", bg="light grey") # this code determines the properties of the search button
        self.searchButton.pack(ipadx=40) # this code creates the search button
        self.searchButton.bind("<Button-1>", self.search) # this code make search func event on search button

        self.resultList = Treeview(self.mainFrame) # this code determines the properties of the resultlist treeview
        self.resultList.bind("<Double-1>", self.on_double_click) # this code make on_double_click func event on resultlist

        self.resultList["columns"] = ("one", "two", "three") # this code determines the how many columns on resultlist
        self.resultList.column("#0", width=200) # this code is for the first column from the left side
        self.resultList.column("one", width=200) # this code is for the second column from the left side
        self.resultList.column("two", width=200) # this code is for the third column from the left side
        self.resultList.column("three", width=200) # this code is for the fourth column from the left side

        self.resultList.heading("#0", text="Rank") # this code determines the first column heading
        self.resultList.heading("one", text="Faculty Member") # this code determines the second column heading
        self.resultList.heading("two", text="Score") # this code determines the third column heading
        self.resultList.heading("three", text="Link") # this code determines the fourth column heading

        self.resultList.pack(fill=X, pady=40) # this code creates the resultlist treeview

class Fetch(GUI): #this code create GUI class

    def __init__(self,parent):

        Frame.__init__(self, parent)# this code ensure that the code we write is in the frame
        self.root = parent
        self.initUI()
        self.dbtables = {"urllist": "urllist.db", "wordlocation": "wordlocation.db", "link": "link.db",# this code create dbtables for using it crawling and searching
                         "linkwords": "linkwords.db", "pagerank": "pagerank.db"}


    def fetch_profiles(self, event):

        self.progressLabel.configure(text="Collecting Faculty Members' Profile Links...") # this code configure progresslabel text
        self.update()# this code update the frame
        driver = webdriver.Chrome("chromedriver.exe") # this code creates driver object
        url = self.urlEntry.get() # this code get the url from the url entry
        driver.get(url) # this code open the given url in chrome
        elem = driver.find_element_by_xpath("//*") # getting all html address
        html_doc = elem.get_attribute("outerHTML")
        soup = BeautifulSoup(html_doc, 'html.parser') # this code is for creating soup object and parse html content
        div = soup.find_all('div', attrs= {'class':'academic-staff-category'}) # this code find items from academic-staff-category class

        academic_staff_links = []
        departments = []

        for part in div: # this for loop get items from the academic-staff-category class
            for link in part.find_all("a"): # this code search link from the a class
                academic_staff_links.append(link.get("href")) # this code get the links and append the link into the list
                department = link.find('span',attrs={'class':'academic-staff-department'}) # this code find items from the academic-staff-department class
                if department.string not in departments: # this code checks if department is not in the departments list
                    departments.append(department.string) # this code append department as string into the departments list

        departments.append("All") # this code append All word to the departments list
        for department in sorted(departments): # get each department in the list
            self.departmentCombobox['values'] = (*self.departmentCombobox['values'], department) # this code add the departments lists items to the department combobox
        self.departmentCombobox.current(0) # this code shows the initial item on the combobox

        academic_staff_links_url = []
        for link in academic_staff_links: # get each academic-staff-link
            a = "https://www.sehir.edu.tr"+link # this code unite the links with url component
            academic_staff_links_url.append(a) # this code add new url to the academic_staff_links_url list

        crawler = mysearchengine.Crawler(self.dbtables) # creates crawler object from mysearchenine
        crawler.createindextables()# we creates database tables for saving datas in these files
        crawler.crawl(academic_staff_links_url, depth=1) # we use crawl method for getting each url's content and saving it to the db

        self.progressLabel.configure(text="Fetching Profiles ...") # this code changes the progress label text
        self.update() # update frame

        input_file = "wordlocation.db.dat"
        file_size = os.path.getsize(input_file) # this code gets the file size
        piece_size = 1024 #1024 byte = 1KB
        step = 400 / float(file_size / piece_size) #400 equal length of progressbar and file_size/piece_size give us how much KB saved in this file
        progress = 0
        while True:
            if progress >= (step * (file_size/piece_size)): # this code checks if progress is done or not
                return
            progress += step # this code add step to the progress
            progress_label = "%"+'%s Completed' % str((progress//4)).split(".")[0] # when the progress increase it changes the progress label percentage
            self.persentaceLabel.configure(text=progress_label) # this code changes the progress label to the progress_label

            self.progressBar.create_rectangle(0, 0, progress, 25, fill="green") # this code fills the progress bar with green while progress continues
            self.update()#updates frame

class Search(Fetch):#creates fetch class
    def __init__(self,parent):

        Frame.__init__(self, parent)
        self.root = parent
        self.initUI()
        self.dbtables = {"urllist": "urllist.db", "wordlocation": "wordlocation.db", "link": "link.db",#creates db tables for searching
                         "linkwords": "linkwords.db", "pagerank": "pagerank.db"}

    def search(self, event):
        self.resultList.delete(*self.resultList.get_children())# clear the result list for further searches
        search = mysearchengine.searcher(self.dbtables) # creates searcher object by using db tables that we use while fetching
        # search.query(self.keywordEntry.get())
        # I copied and pasted this part because I need to get variable from this part and add loop for checking department.
        results, words = search.getmatchingpages(self.keywordEntry.get())
        if len(results) == 0:
            print('No matching pages found!')
            return

        scores = search.getscoredlist(results, words)
        rankedscores = sorted([(score, url) for (url, score) in scores.items()], reverse=1)
        a = 1
        for (score, url) in rankedscores[0:10]:
            print('%f\t%s' % (score, url))
            r = requests.get(url) # get matched url's
            html_doc = r.content # get content
            soup = BeautifulSoup(html_doc,"html.parser") # creates soup object
            name = soup.find(class_="name") # get academic staff name in the content
            department = soup.find(class_="departmant-bolum") # get academic staff department
            if self.departmentCombobox.get() == "All": # check the combobox value equal to the all
                self.resultList.insert("","end",text=str(a), values=(name.text.split(".")[-1], round(score,2) , url))# insert result to the result list
                a += 1
            elif self.departmentCombobox.get() == department.text: #check the academic staff department equal to the combobox value
                self.resultList.insert("","end",text=str(a), values=(name.text.split(".")[-1], round(score,2) , url)) # insert result to the list
                a += 1
            else:
                pass

    def on_double_click(self,event):

        clicked_column = self.resultList.identify_column(event.x) # get clicked column number ex:#2
        values_row = self.resultList.item(self.resultList.focus()) # get clicked rows informations
        column_number = int(clicked_column.split("#")[1]) #get only number
        link = values_row["values"][column_number-1] #get the url from row values
        driver = webdriver.Chrome("chromedriver.exe") #creates driver
        driver.get(link) # open double clicked link on chrome

def main():
    root = Tk()
    root.title("Search Engine") # this code makes the title of our root title as Search Engine
    root.geometry("900x800") # this code makes the root geometry
    app = Search(root) # this code creates Search class object by using tkinter root
    app.pack(fill=BOTH, expand=True) # this code creates the app
    root.mainloop()
main() # run main function